### Execution Plans and the effect of Caching
Unlike a regular pythong program, map/reduce commands do not always perform any computation when they are executed. Instead, they construct something called an **execution plan**. Only when a result is needed does the computation start. This approach is also called **lazy execution**.

The benefit from lazy execution is in minimizing the the number of memory accesses. Consider for example the following map/reduce commands:
```python
A=RDD.map(lambda x:x*x).filter(lambda x: x%2==0)
A.reduce(lambda x,y:x+y) 
```

The commands defines the following plan. For each number `x` in the RDD:
1. Compute the square of `x`
2. Filter out `x*x` whose value is odd.
3. Sum the elements that were not filtered out.

A naive execution plan is to square all items in the RDD, store the results in a new RDD, then perform a filtering pass, generating a second RDD,  and finally perform the summation. Doing this will require iterating through the RDD three times, and creating 2 interim RDDs. As memory access is the bottleneck in this type of computation, the execution plan is slow.

A better execution plan is to perform all three operations on each element of the RDD in sequence, and then move to the next element. This plan is faster because we iterate through the elements of the RDD only once, and because we don't need to save the intermediate results. We need to maintain only one variable: the partial sum, and as that is a single variable, we can use a CPU register.

#### The `%%time` magic

The `%%time` command is a *cell magic* which measures the execution time of the cell. We will mostly be interested in the wall time, which includes the time it takes to move data in the memory hierarchy.

([More about ipython magics](https://ipython.org/ipython-doc/3/interactive/magics.html))

### Preparations
In the following cells we create an RDD and define a function which wastes some time and then returns `cos(i)`. We want the function to waste some time so that the time it takes to compute the `map` operation is significant.

In [15]:
%%time
# create an RDD with one million elements.
RDD=sc.parallelize(range(1000000))

CPU times: user 67.3 ms, sys: 30.8 ms, total: 98.1 ms
Wall time: 111 ms


In [2]:
from math import cos
def taketime(i):
    [cos(j) for j in range(10)]
    return cos(i)

In [3]:
%%time
taketime(5)

CPU times: user 14 µs, sys: 1 µs, total: 15 µs
Wall time: 20 µs


0.2836621854632263

### Cascading operations

In the next cell we define the `map` operation. Note that this cell takes very little time.

In [4]:
%%time
Interm=RDD.map(lambda x: taketime(x))

CPU times: user 26 µs, sys: 19 µs, total: 45 µs
Wall time: 42.2 µs


### Execution plan
The reason that the previous cell took less than 50 micro-seconds (Wall Time) is that no computation was done. The cell defined the 'map' action, but did not execute it yet.

**Lazy Execution** refers to this type of behaviour. The system delays actual computation until the latest possible moment. Instead of computing the content of the RDD, it adds the RDD to the **execution plan**.

Using Lazy evaluation of a plan has two main advantages relative to immediate execution of each step:
1. A single pass over the data, rather than multiple passes.
2. Smaller memory footprint becase no intermediate results are saved.

At this point the variable `Interm` does not point to an actual data structure. Instead, it points to an execution plan expressed as a **dependence graph**. The dependence graph defines the dependence of the RDD on each other.

The dependence graph associated with an RDD can be printed out using the method `toDebugString()`.
The first line corresponds to `Interm` and the second line corresponds to `RDD` which is the input to interm
```
(2) PythonRDD[8] at RDD at PythonRDD.scala:43 []
 |  ParallelCollectionRDD[4] at parallelize at PythonRDD.scala:423 []
```

In [16]:
print Interm.toDebugString()

(2) PythonRDD[4] at RDD at PythonRDD.scala:43 [Memory Serialized 1x Replicated]
 |       CachedPartitions: 2; MemorySize: 8.6 MB; ExternalBlockStoreSize: 0.0 B; DiskSize: 0.0 B
 |  ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:423 [Memory Serialized 1x Replicated]


### Actual execution
The `reduce` command needs to output an actual output, **spark** therefor has to actually execute the `map` and the `reduce`. Some real computation needs to be done, which takes about 1.78 seconds (Wall time).

In [6]:
%%time
print 'out=',Interm.reduce(lambda x,y:x+y)

out= -0.288705467969
CPU times: user 9.82 ms, sys: 4.06 ms, total: 13.9 ms
Wall time: 2.68 s


### Executing a different calculation based on the same plan.
The plan defined by `Interm` can be executed many times. Below we give an example. 

**Note:** the run-time is similar to that of the previous command because the intermediate results that are due to `Interm` have not been saved in memory.

In [7]:
%%time
print 'out=',Interm.filter(lambda x:x>0).count()

out= 500000
CPU times: user 8.71 ms, sys: 3.13 ms, total: 11.8 ms
Wall time: 1.91 s


### Caching intermediate results
The computation above is wasteful because each time we recompute the `map` operation.

We sometimes want to keep the intermediate results in memory so that we can reuse them without recalculating them. This will reduce the running time, at the cost of requiring more memory.

The method `cache()` indicates that the RDD generates in this plan should be stored in memory. Note that this is still only a *plan*. The actual calculation will be done only when the final result is seeked.

In [8]:
%%time
Interm=RDD.map(lambda x: taketime(x)).cache()

CPU times: user 3.75 ms, sys: 1.69 ms, total: 5.44 ms
Wall time: 10.1 ms


### Plan to cache
The definition of `Interm` is almost the same as before. However, the *plan* corresponding to `Interm` is more elaborate and contains information about how the intermediate results will be chached and partitioned.

In [9]:
print Interm.toDebugString()

(2) PythonRDD[4] at RDD at PythonRDD.scala:43 [Memory Serialized 1x Replicated]
 |  ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:423 [Memory Serialized 1x Replicated]


### Creating the cache
The following command executes the first map-reduce command **and** caches the result of the `map` command in memory. 

In [10]:
%%time
print 'out=',Interm.reduce(lambda x,y:x+y)

out= -0.288705467969
CPU times: user 6.02 ms, sys: 3.11 ms, total: 9.13 ms
Wall time: 1.81 s


### Using the cache
This time `Interm` is cached. Therefor the second use of `Interm` is much faster than when we did not use `cache`: 0.25 second instead of 1.9 second. (your milage may vary depending on the computer you are running this on).

In [11]:
%%time
print 'out=',Interm.filter(lambda x:x>0).count()

out= 500000
CPU times: user 7.1 ms, sys: 3.06 ms, total: 10.2 ms
Wall time: 244 ms
